In [1]:
# Libraries & directories
import pandas as pd

In [2]:
dirData = '../data/'
dirDataExt = dirData + 'external/'

dirCensus = dirDataExt + 'census/'

In [3]:
# 1990s first
f1990 = dirCensus + 'crhia90.txt'

# 1990's Census is fixed width and weird -> columns and names
f1990Cols = [(0,6), (6,40), (40, 51), (51, 60), (60, 69), (69, 78), (78, 86), (86, 96), (96, 104), (105, 114)]
f1990ColNames = ['FIPS', 'County Name', 'Total Population', 'White', 'White - Hispanic', 'White - non hispanic', 'Black', 'American Indian & Native Alaskan', 'Asian & Pacific Islander', 'Hispanic']

# The file has 8 rows of metadata
# use cols and header, set header to none for custom cols
df1990 = pd.read_fwf(f1990, skiprows=16, colspecs=f1990Cols,
                     header=None, names=f1990ColNames, index_col=False, 
                     nrows=99)

# Calc the percentage
df1990['hisp_perc'] = df1990['Hispanic'] / df1990['Total Population']

In [4]:
df1990.head()

,FIPS,County Name,Total Population,White,White - Hispanic,White - non hispanic,Black,American Indian & Native Alaskan,Asian & Pacific Islander,Hispanic,hisp_perc
0,19001,Adair County,8409,8384,28,8356,1,5,19,35,0.004162
1,19003,Adams County,4866,4853,16,4837,3,7,3,18,0.003699
2,19005,Allamakee County,13855,13798,32,13766,5,21,31,42,0.003031
3,19007,Appanoose County,13743,13601,71,13530,79,27,36,72,0.005239
4,19009,Audubon County,7334,7324,21,7303,1,3,6,22,0.003000


In [5]:
# 1995 next
f1995 = dirCensus + 'crhia95.txt'

f1995Cols = f1900Cols = [(0,6), (6,40), (40, 51), (51, 60), (60, 69), (69, 78), (78, 86), (86, 96), (96, 105), (105, 115)]

# Reuse the 1990 cols & names
df1995 = pd.read_fwf(f1995, skiprows=16, colspecs=f1995Cols,
                     header=None, names=f1990ColNames, index_col=False, 
                    nrows=99)

# Calc the percentage
df1995['hisp_perc'] = df1995['Hispanic'] / df1995['Total Population']

In [6]:
df1995.head()

,FIPS,County Name,Total Population,White,White - Hispanic,White - non hispanic,Black,American Indian & Native Alaskan,Asian & Pacific Islander,Hispanic,hisp_perc
0,19001,Adair County,8246,8216,36,8180,1,4,25,47,0.005700
1,19003,Adams County,4507,4491,20,4471,5,8,3,22,0.004881
2,19005,Allamakee County,14134,14016,43,13973,49,26,43,65,0.004599
3,19007,Appanoose County,13598,13451,88,13363,78,28,41,89,0.006545
4,19009,Audubon County,6894,6883,24,6859,2,3,6,26,0.003771


In [10]:
# 2000
f2000s = dirCensus + 'CO-EST00INT-SEXRACEHISP.csv'

df2000s = pd.read_csv(f2000s, dtype={'COUNTY':str}, encoding = 'iso-8859-1')

In [11]:
# filtering...
# SEX = 0 [total]
# RACE = 0 [all races combined]
# ORIGIN = {0:'Total', 1:'Not Hispanic', 2:'Hispanic'}

# State=19 at the end

filt2000s = df2000s[['STATE', 'COUNTY', 'CTYNAME', 'SEX', 'ORIGIN', 'RACE', 'POPESTIMATE2000', 'POPESTIMATE2005', 'POPESTIMATE2010']][(df2000s.STATE == 19)]
filt2000s.head()

,STATE,COUNTY,CTYNAME,SEX,ORIGIN,RACE,POPESTIMATE2000,POPESTIMATE2005,POPESTIMATE2010
49707,19,001,Adair County,0,0,0,8223,7932,7675
49708,19,001,Adair County,0,0,1,8163,7865,7587
49709,19,001,Adair County,0,0,2,6,10,30
49710,19,001,Adair County,0,0,3,6,4,7
49711,19,001,Adair County,0,0,4,20,19,27


In [12]:
# 2000

# Create another filtered dataframe with only the 2000's info
hisp2000 = filt2000s[['CTYNAME', 'COUNTY', 'SEX', 'ORIGIN', 'RACE', 'POPESTIMATE2000']]

# create the final dataframe
# rename column to hispanic population
df2000 = hisp2000[['CTYNAME', 'COUNTY', 'POPESTIMATE2000']][(hisp2000.SEX == 0) & (hisp2000.RACE == 0) & (hisp2000.ORIGIN == 2)].rename(columns={'POPESTIMATE2000': 'hisp_pop'}) 

# Add column, but use the indexes to assign value
df2000['total_pop'] = hisp2000['POPESTIMATE2000'][(hisp2000.SEX == 0) & (hisp2000.RACE == 0) & (hisp2000.ORIGIN == 0)].values

# Finally, create the percentage
df2000['hisp_perc'] = df2000['hisp_pop'] / df2000['total_pop']

In [13]:
# 2005

# Create another filtered dataframe with only the 2000's info
hisp2005 = filt2000s[['CTYNAME', 'COUNTY', 'SEX', 'ORIGIN', 'RACE', 'POPESTIMATE2005']]

# rename column to hispanic population
df2005 = hisp2005[['CTYNAME', 'COUNTY', 'POPESTIMATE2005']][(hisp2005.SEX == 0) & (hisp2005.RACE == 0) & (hisp2005.ORIGIN == 2)].rename(columns={'POPESTIMATE2005': 'hisp_pop'}) 

# Add column, but use the indexes to assign value
df2005['total_pop'] = hisp2005['POPESTIMATE2005'][(hisp2005.SEX == 0) & (hisp2005.RACE == 0) & (hisp2005.ORIGIN == 0)].values

# Finally, create the percentage
df2005['hisp_perc'] = df2005['hisp_pop'] / df2005['total_pop']

In [14]:
# 2010

# Create another filtered dataframe with only the 2000's info
hisp2010 = filt2000s[['CTYNAME', 'COUNTY', 'SEX', 'ORIGIN', 'RACE', 'POPESTIMATE2010']]

# rename column to hispanic population
df2010 = hisp2010[['CTYNAME', 'COUNTY', 'POPESTIMATE2010']][(hisp2010.SEX == 0) & (hisp2010.RACE == 0) & (hisp2010.ORIGIN == 2)].rename(columns={'POPESTIMATE2010': 'hisp_pop'}) 

# Add column, but use the indexes to assign value
df2010['total_pop'] = hisp2010['POPESTIMATE2010'][(hisp2010.SEX == 0) & (hisp2010.RACE == 0) & (hisp2010.ORIGIN == 0)].values

# Finally, create the percentage
df2010['hisp_perc'] = df2010['hisp_pop'] / df2010['total_pop']

In [15]:
df2010.head()

,CTYNAME,COUNTY,hisp_pop,total_pop,hisp_perc
49721,Adair County,001,94,7675,0.012248
49784,Adams County,003,44,4019,0.010948
49847,Allamakee County,005,729,14355,0.050784
49910,Appanoose County,007,189,12877,0.014677
49973,Audubon County,009,45,6107,0.007369


In [24]:
# 2015

f2015 = dirCensus + 'CC-EST2015-ALLDATA-19.csv'

df2015 = pd.read_csv(f2015, dtype={'COUNTY':str})

# 2015 file has ALL counties, we'll need to filter down
# YEAR = 8 [2015 estimate]
# AGEGRP = 0 [Total]
df2015 = df2015[(df2015.YEAR == 8) & (df2015.AGEGRP == 0)]

# add total hispanic population
df2015['hisp_pop'] = df2015.apply(lambda x: x.H_MALE + x.H_FEMALE, axis=1)

# And calculate the percentage
df2015['hisp_perc'] = df2015.hisp_pop / df2015.TOT_POP

In [25]:
df2015.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,hisp_pop,hisp_perc
133,50,19,001,Iowa,Adair County,8,0,7228,3553,3675,...,4,4,5,2,3,2,1,0,118,0.016325
285,50,19,003,Iowa,Adams County,8,0,3796,1897,1899,...,1,3,1,1,0,1,0,0,41,0.010801
437,50,19,005,Iowa,Allamakee County,8,0,13886,7079,6807,...,6,12,34,21,4,5,10,16,769,0.055380
589,50,19,007,Iowa,Appanoose County,8,0,12529,6187,6342,...,8,5,6,5,4,3,6,1,209,0.016681
741,50,19,009,Iowa,Audubon County,8,0,5773,2822,2951,...,2,7,0,5,0,0,0,0,58,0.010047


In [27]:
# Canonical dataframe

# Create a dataframe from 2010 and keep only the vars we want
dfCanonical = df2010[['CTYNAME', 'COUNTY', 'hisp_perc']].rename(columns={'hisp_perc':'hisp_perc2010'})

# Assign and rename percentages calculated from previous dataframes
dfCanonical['hisp_perc2005'] = df2005['hisp_perc']
dfCanonical['hisp_perc2000'] = df2000['hisp_perc']

# indices mismatch, just use values as all are in order
dfCanonical['hisp_perc1995'] = df1995['hisp_perc'].values
dfCanonical['hisp_perc1990'] = df1990['hisp_perc'].values
dfCanonical['hisp_perc2015'] = df2015['hisp_perc'].values


dfCanonical.head()

,CTYNAME,COUNTY,hisp_perc2010,hisp_perc2005,hisp_perc2000,hisp_perc1995,hisp_perc1990,hisp_perc2015
49721,Adair County,001,0.012248,0.009834,0.006932,0.005700,0.004162,0.016325
49784,Adams County,003,0.010948,0.007099,0.005809,0.004881,0.003699,0.010801
49847,Allamakee County,005,0.050784,0.050310,0.035369,0.004599,0.003031,0.055380
49910,Appanoose County,007,0.014677,0.011893,0.009930,0.006545,0.005239,0.016681
49973,Audubon County,009,0.007369,0.004586,0.004850,0.003771,0.003000,0.010047


In [29]:
# write out to a csv
fCanonical = dirData + 'processed/populations/ia-hisp-counties.csv'
dfCanonical.to_csv(fCanonical, float_format='%.3f', index=False)